# Hyperparameter tuning script for ML models

### Example scripts designed by SiHun Lee, Ph. D using LLM API and Meshgraphnets

In [ ]:
import httpx
import json
from pathlib import Path
from datetime import datetime
import time
from IPython.display import display, Latex

# Build Universal LLM API Client
class LLMApiClient:
    def __init__(self, base_url: str, timeout: float = 3600000000.0):
        self.base_url = base_url.rstrip("/")
        self.token = None
        self.timeout = httpx.Timeout(50.0, read=timeout, write=timeout, pool=timeout)

    def _headers(self):
        return {"Authorization": f"Bearer {self.token}"} if self.token else {}

    def login(self, username: str, password: str):
        r = httpx.post(f"{self.base_url}/api/auth/login", 
                      json={"username": username, "password": password}, timeout=10.0)
        r.raise_for_status()
        self.token = r.json()["access_token"]
        return r.json()

    def list_models(self):
        r = httpx.get(f"{self.base_url}/v1/models", headers=self._headers(), timeout=10.0)
        r.raise_for_status()
        return r.json()

    def chat_new(self, model: str, user_message: str, agent_type: str = "auto", files: list = None):
        messages = [{"role": "user", "content": user_message}]
        data = {"model": model, "messages": json.dumps(messages), "agent_type": agent_type}
        
        files_to_upload = []
        if files:
            for file_path in files:
                f = open(file_path, "rb")
                files_to_upload.append(("files", (Path(file_path).name, f)))
        
        try:
            r = httpx.post(f"{self.base_url}/v1/chat/completions", data=data,
                          files=files_to_upload if files_to_upload else None,
                          headers=self._headers(), timeout=self.timeout)
            r.raise_for_status()
            result = r.json()
            return result["choices"][0]["message"]["content"], result["x_session_id"]
        finally:
            for _, (_, f) in files_to_upload:
                f.close()

    def chat_continue(self, model: str, session_id: str, user_message: str, 
                     agent_type: str = "auto", files: list = None):
        messages = [{"role": "user", "content": user_message}]
        data = {"model": model, "messages": json.dumps(messages), 
                "session_id": session_id, "agent_type": agent_type}
        
        files_to_upload = []
        if files:
            for file_path in files:
                f = open(file_path, "rb")
                files_to_upload.append(("files", (Path(file_path).name, f)))
        
        try:
            r = httpx.post(f"{self.base_url}/v1/chat/completions", data=data,
                          files=files_to_upload if files_to_upload else None,
                          headers=self._headers(), timeout=self.timeout)
            r.raise_for_status()
            result = r.json()
            return result["choices"][0]["message"]["content"], result["x_session_id"]
        finally:
            for _, (_, f) in files_to_upload:
                f.close()

    def get_session_artifacts(self, session_id: str):
        """Get list of files generated during the session"""
        r = httpx.get(f"{self.base_url}/api/chat/sessions/{session_id}/artifacts",
                     headers=self._headers(), timeout=10.0)
        r.raise_for_status()
        return r.json()

    def download_artifact(self, session_id: str, filename: str, save_to: str = None):
        """
        Download a generated artifact file to local disk.
        
        Args:
            session_id: The session ID that generated the file
            filename: Name of the file to download (can include subdirectory, e.g., 'temp_charts/chart.png')
            save_to: Local path to save the file (default: current directory with original filename)
        
        Returns:
            str: Path to the downloaded file
        
        Example:
            client.download_artifact(session_id, "Warpage_Report_20250126.pptx", "./downloads/report.pptx")
        """
        r = httpx.get(
            f"{self.base_url}/api/chat/sessions/{session_id}/artifacts/{filename}",
            headers=self._headers(),
            timeout=60.0
        )
        r.raise_for_status()
        
        # Determine local save path
        if save_to is None:
            save_to = Path(filename).name  # Use just the filename, not subdirectory
        
        # Create parent directories if needed
        save_path = Path(save_to)
        save_path.parent.mkdir(parents=True, exist_ok=True)
        
        # Write file content
        with open(save_path, "wb") as f:
            f.write(r.content)
        
        return str(save_path)

# Configuration
API_BASE_URL = 'http://10.198.112.203:10007'
USERNAME = "ppt"
PASSWORD = "ppt"

# Initialize and login
client = LLMApiClient(API_BASE_URL, timeout=36000000.0)# 10 hours
client.login(USERNAME, PASSWORD)
models = client.list_models()
MODEL = models["data"][0]["id"]

print(f"✓ Logged in as: {USERNAME}")
print(f"✓ Using model: {MODEL}")

In [ ]:
import os

# ML CODE DIRECTORY (expanduser handles ~ expansion)
mother_dir = os.path.expanduser('~/scratch1/MeshGraphNets')
# docs directory
docs_path = os.path.join(mother_dir, 'CONFIG_AND_EXECUTION_GUIDE.md')

# Now make the LLM API read the docs and run the code
prompt = f"""
You are executing a long-running hyperparameter tuning experiment for ML models.

## Your Task
1. Read the attached documentation file carefully
2. Understand the configuration options and execution workflow
3. Plan out the hyperparameter tuning experiment
4. Build configuration file for the experiment with distinguishable output file
5. Execute the ML code as described in the documentation parallelly

## Important Requirements
- **Working Directory**: All code should run from `{mother_dir}`
- **Logging**: Use the log file implemented in the code and with timestamps, distinguish filenames with config
- **Error Handling**: If a single hyperparameter combination fails, log the error and continue with the next combination
- **Results**: Save final results to a CSV/JSON file with all hyperparameter combinations and their metrics

## Execution Guidelines
- Use `cd {mother_dir}` at the start of your Python code
- Do NOT stop until all hyperparameter combinations are tested
- At the end, provide a summary of the best hyperparameters found

Begin by reading the documentation, then execute the training.
"""

# Send with the docs file attached
response, session_id = client.chat_new(
    model=MODEL,
    user_message=prompt,
    agent_type="react",  # Use react agent for tool calling
    files=[docs_path]
)

# Save session for recovery (important for multi-day runs)
with open("active_session.txt", "w") as f:
    f.write(session_id)
print(f"Session ID saved: {session_id}")
print(response)

In [ ]:
# Check progress of ongoing experiment
# Uncomment the line below to load session from file if kernel restarted
# session_id = open("active_session.txt").read().strip()

check_prompt = """
Report current progress:
1. How many hyperparameter combinations completed?
2. Current best result so far?
3. Any errors encountered?
4. Estimated remaining time?
"""

response, _ = client.chat_continue(
    model=MODEL,
    session_id=session_id,
    user_message=check_prompt,
    agent_type="react"
)
print(response)

In [ ]:
# Download results after completion
# Uncomment the line below to load session from file if kernel restarted
# session_id = open("active_session.txt").read().strip()

artifacts = client.get_session_artifacts(session_id)
print("Available files:", artifacts)

# Download results (CSV, JSON, and log files)
import os
os.makedirs("./results", exist_ok=True)

for artifact in artifacts.get("files", []):
    if artifact.endswith(('.csv', '.json', '.log')):
        local_path = client.download_artifact(session_id, artifact, f"./results/{artifact}")
        print(f"Downloaded: {local_path}")